<a href="https://colab.research.google.com/github/AvellinaLeong/NHM-Nannofossil-Segmentation-Project/blob/main/01b_Save_Test_Segmentations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Save Individual Segmentations to Output Folder



*   Save segmentation instances in both training and validation sets, generated from trained model
*   Use these instances to run a binary classification model



In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# Set script location to own development space
MY_DEVELOPMENT_SPACE = '/content/drive/MyDrive/development/avellina/'
import os
os.chdir(MY_DEVELOPMENT_SPACE)
!pwd
!ls

Mounted at /content/drive/
/content/drive/MyDrive/development/avellina
Binary_Classification_notebooks  Detectron2_notebooks  Morphometrics_notebooks	src
detectron2			 Mask-RCNN	       output


In [ ]:
project_dir = "/content/drive/MyDrive/data/species_53"

## Load Detectron2 and Model

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 16.4 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
fatal: destination path 'detectron2' already exists and is not an empty directory.
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.3 ; cuda:  cu121
detectron2: 0.6


In [ ]:
# Import Detectron2 and Detectron2 Logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

In [ ]:
import os
import cv2
import numpy as np
import yaml
from PIL import Image
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

## Load Configuration and Train+Val Sets

In [ ]:
# Load the saved configuration from the YAML file
config_yaml_path = "/content/drive/MyDrive/data/species_53/Detectron2_Models/1/config_1.yaml"  # Path to your saved config file
cfg = get_cfg()

# Set default values for keys that might be missing in the config file
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.VAL = ("my_dataset_val",)
cfg.DATASETS.TEST = ("my_dataset_test",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = ""
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.TEST.EVAL_PERIOD = 500
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

# Merge the configuration file with the default (debugged this way)
cfg.merge_from_file(config_yaml_path)
# Initialize the predictor with the loaded configuration
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "1", "model_final_1.pth")

In [ ]:
# Create the predictor
predictor = DefaultPredictor(cfg)

[08/05 22:51:38 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/data/species_53/Detectron2_Models/1/model_final_1.pth ...


In [ ]:
# Register train and val datasets in COCO format
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_test", {}, "/content/drive/MyDrive/data/species_53/data/test/coco_test.json", "/content/drive/MyDrive/data/species_53/data/test")

In [ ]:
# Extract the metadata and dataset dictionaries for train, test and val
test_metadata = MetadataCatalog.get("my_dataset_test")
test_dataset_dicts = DatasetCatalog.get("my_dataset_test")

[08/05 22:51:57 d2.data.datasets.coco]: Loading /content/drive/MyDrive/data/species_53/data/test/coco_test.json takes 1.77 seconds.
[08/05 22:51:57 d2.data.datasets.coco]: Loaded 96 images in COCO format from /content/drive/MyDrive/data/species_53/data/test/coco_test.json


## Crop the segmentation instances and save to output directory

In [ ]:
# Create the output directory for segmented test images
output_dir = "/content/drive/MyDrive/data/species_53/Binary_Classification/cropped_segmentations"
test_output_dir = os.path.join(output_dir, "test")
os.makedirs(test_output_dir, exist_ok=True)

In [ ]:
# Function to process and save segmented instances
def process_and_save_segmented_images(dataset_dicts, output_subdir):
    for d in dataset_dicts:
        im = cv2.imread(d["file_name"])
        outputs = predictor(im)

        # Get the masks and bounding boxes
        masks = outputs["instances"].pred_masks.to("cpu").numpy()
        num_instances = masks.shape[0]

        for i in range(num_instances):
            # Get the mask for the current instance
            item_mask = masks[i]

            # Get the true bounding box of the mask
            segmentation = np.where(item_mask == True)
            x_min = int(np.min(segmentation[1]))
            x_max = int(np.max(segmentation[1]))
            y_min = int(np.min(segmentation[0]))
            y_max = int(np.max(segmentation[0]))

            # Crop the original image using the bounding box coordinates
            cropped = Image.fromarray(im[y_min:y_max, x_min:x_max, :], mode='RGB')

            # Construct the output file path
            base_name = os.path.basename(d["file_name"])
            file_name, file_ext = os.path.splitext(base_name)
            output_file_name = f"{file_name}_{i+1}{file_ext}"   # make sure to give unique name so doesn't rewrite
            output_path = os.path.join(output_subdir, output_file_name)

            # Save the cropped image
            cropped.save(output_path)

# Process and save segmented instances for train and val datasets
process_and_save_segmented_images(test_dataset_dicts, test_output_dir)

print("Segmented instances saved successfully for test dataset.")

/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Segmented instances saved successfully for train and val datasets.
